In [1]:
import pandas as pd

def add_code(df, d_code, h_code, l_code):
    df = df.copy()   

    # D Code
    df['person_prefer_d_1_n'] = df['person_prefer_d_1'].apply(lambda x: d_code[x]['속성 D 세분류코드'])
    df['person_prefer_d_1_s'] = df['person_prefer_d_1'].apply(lambda x: d_code[x]['속성 D 소분류코드'])
    df['person_prefer_d_1_m'] = df['person_prefer_d_1'].apply(lambda x: d_code[x]['속성 D 중분류코드'])
    df['person_prefer_d_1_l'] = df['person_prefer_d_1'].apply(lambda x: d_code[x]['속성 D 대분류코드'])

    df['person_prefer_d_2_n'] = df['person_prefer_d_2'].apply(lambda x: d_code[x]['속성 D 세분류코드'])
    df['person_prefer_d_2_s'] = df['person_prefer_d_2'].apply(lambda x: d_code[x]['속성 D 소분류코드'])
    df['person_prefer_d_2_m'] = df['person_prefer_d_2'].apply(lambda x: d_code[x]['속성 D 중분류코드'])
    df['person_prefer_d_2_l'] = df['person_prefer_d_2'].apply(lambda x: d_code[x]['속성 D 대분류코드'])

    df['person_prefer_d_3_n'] = df['person_prefer_d_3'].apply(lambda x: d_code[x]['속성 D 세분류코드'])
    df['person_prefer_d_3_s'] = df['person_prefer_d_3'].apply(lambda x: d_code[x]['속성 D 소분류코드'])
    df['person_prefer_d_3_m'] = df['person_prefer_d_3'].apply(lambda x: d_code[x]['속성 D 중분류코드'])
    df['person_prefer_d_3_l'] = df['person_prefer_d_3'].apply(lambda x: d_code[x]['속성 D 대분류코드'])

    df['contents_attribute_d_n'] = df['contents_attribute_d'].apply(lambda x: d_code[x]['속성 D 세분류코드'])
    df['contents_attribute_d_s'] = df['contents_attribute_d'].apply(lambda x: d_code[x]['속성 D 소분류코드'])
    df['contents_attribute_d_m'] = df['contents_attribute_d'].apply(lambda x: d_code[x]['속성 D 중분류코드'])
    df['contents_attribute_d_l'] = df['contents_attribute_d'].apply(lambda x: d_code[x]['속성 D 대분류코드'])

    # H Code
    df['person_prefer_h_1_u'] = df['person_prefer_h_1'].apply(lambda x: h_code[x]['속성 H 상위코드'])
    df['person_prefer_h_2_u'] = df['person_prefer_h_2'].apply(lambda x: h_code[x]['속성 H 상위코드'])
    df['person_prefer_h_3_u'] = df['person_prefer_h_3'].apply(lambda x: h_code[x]['속성 H 상위코드'])
    df['contents_attribute_h_u'] = df['contents_attribute_h'].apply(lambda x: h_code[x]['속성 H 상위코드'])

    # L Code
    df['contents_attribute_l_n'] = df['contents_attribute_l'].apply(lambda x: l_code[x]['속성 L 세분류코드'])
    df['contents_attribute_l_s'] = df['contents_attribute_l'].apply(lambda x: l_code[x]['속성 L 소분류코드'])
    df['contents_attribute_l_m'] = df['contents_attribute_l'].apply(lambda x: l_code[x]['속성 L 중분류코드'])
    df['contents_attribute_l_l'] = df['contents_attribute_l'].apply(lambda x: l_code[x]['속성 L 대분류코드'])
    return df

In [3]:
d_code = pd.read_csv('D_code.csv', index_col=0).T.to_dict()
h_code = pd.read_csv('H_code.csv', index_col=0).T.to_dict()
l_code = pd.read_csv('L_code.csv', index_col=0).T.to_dict()

df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

df_train = add_code(df_train, d_code, h_code, l_code)
df_test = add_code(df_test, d_code, h_code, l_code)

In [12]:
df_train.drop(['person_prefer_g', 'person_prefer_f'], axis=1, inplace=True)
df_test.drop(['person_prefer_g', 'person_prefer_f'], axis=1, inplace=True)

In [11]:
df_train['contents_open_dt'] = pd.to_datetime(df_train['contents_open_dt'])
df_test['contents_open_dt'] = pd.to_datetime(df_test['contents_open_dt'])

In [13]:
def split_datetime(df) :
#     df['year'] = df['contents_open_dt'].dt.year
    df['month'] = df['contents_open_dt'].dt.month
    df['day'] = df['contents_open_dt'].dt.day
    df['hour'] = df['contents_open_dt'].dt.hour
    df['minute'] = df['contents_open_dt'].dt.minute
    return df

In [14]:
df_train = split_datetime(df_train)
df_test = split_datetime(df_test)

In [18]:
df_train_x = df_train.drop(['contents_open_dt', 'target'], axis=1)
df_train_y = df_train[['target']]

df_test_x = df_test.drop(['contents_open_dt'], axis=1)

In [21]:
from sklearn.model_selection import train_test_split

new_train_x, new_val_x, new_train_y, new_val_y = train_test_split(df_train_x, df_train_y, test_size=0.3 ,random_state=777)

In [22]:
import lightgbm as lgb
from sklearn.metrics import accuracy_score

params = {
    'learning_rate' : 0.05,
        'max_depth' : 128,
    'objective' : 'binary',
    'metric' : 'binary_logloss',
    'is_training_metric' : True,
    'num_leave' : 64,
    'feature_fraction' : 0.7,
    'bagging_fraction' : 0.8,
    'bagging_freq':5,
    'seed':2020
}

train_dataset = lgb.Dataset(new_train_x, label=new_train_y)
val_dataset = lgb.Dataset(new_val_x, label=new_val_y)

model = lgb.train(params, train_dataset, 2000, val_dataset, verbose_eval=200, early_stopping_rounds=100)

pred_y = model.predict(new_val_x)
pred_y = [1 if i>0.5 else 0 for i in pred_y]
accuracy_score(new_val_y, pred_y)  
# 0.6171755674498294
# 0.6222225173654922
# 0.6269440718260662 -> 0.64806  // 2021-12-23 14:54
# 0.6292749658002736 

# 0.6118177770033494

# 0.6306296734092147  -> 2021-12-28 15:25:05	0.644898965

# 0.6308421765635583

/usr/local/lib/python3.6/dist-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/usr/local/lib/python3.6/dist-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] Unknown parameter: num_leave
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Unknown parameter: num_leave
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Number of positive: 175481, number of negative: 175884
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.280850 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 5225
[LightGBM] [Info] Number of data points in the train set: 351365, number of used features: 59
[LightGBM] [Warning] Unknown parameter: num_leave
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] [binary:Boos

0.7267807100261644

In [26]:
pred_y = model.predict(df_test_x)
pred_y = [1 if i>0.5 else 0 for i in pred_y]
submission_df['target'] = pred_y
submission_df.to_csv('sub12.csv', index=False)

In [24]:
submission_df = pd.read_csv('sample_submission.csv')

In [27]:
submission_df

,id,target
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0
...,...,...
46399,46399,0
46400,46400,0
46401,46401,0
46402,46402,0
